BB84 Quantum Cryptography Algorithm for Quantum Key Distribution in an hostile environment

We define 3 quantum circuits: one for the Sender (Alice), one for the Reciever (Bob) and one for the Eavesdropper (Eve) to simulate 3 different quantum computers. Our objective is to simulate what happens 
during a communication exchange between 2 quantum computers, and especially when a third party tries
to intercept their communication to steal data.

We assume that Alice and Bob will use 2 channels for their communication: one public or "unreliable" 
channel where they will exchange their messages, and one private "safer" one where they will exchange the 
private quantum key extracted from their previous public communication.
If Eve attempts to spy their communication on the public channel, Alice and Bob should be able to notice 
her, due to the fact that their keys will differ. In that case, they will proceed to change 
their public communication channel.


In [3]:
# Library Imports:

from qiskit import QuantumCircuit, execute, BasicAer
import numpy as np

In [72]:
# Functions Definition:

def encode(circuit, state, basis): 
    
    # Sender prepares qubits
    for i in range(len(basis)):
        if state[i] == 1:   # the quantum circuit starts with all qubits to 0, so we apply "not" to encode 1
            circuit.x(i)
        if basis[i] == 1:   # if the basis is 1 we encode through Hadamard, otherwise we leave it Computational
            circuit.h(i)    # so, when the basis is 0, the qubits are encoded with a Computational basis and result
                            # in the same value of the original state, while when the basis is 1 the qubits are proabilistic
    
    return circuit

def process(circuit, measurement_basis):
    
    # Decryption
    for i in range(len(measurement_basis)):
        if measurement_basis[i] == 1:   # when the measurement basis is 1, we decrypt with hadamard, otherwise we leave it as is
            circuit.h(i)
    
    return circuit

def measure_state(circuit, num_qubits, name):
    circuit.measure_all() # last thing to be done
    back = BasicAer.get_backend('qasm_simulator')
    key = execute(circuit.reverse_bits(),backend=back,shots=1).result().get_counts().most_frequent()
    state = np.zeros(num_qubits, dtype = int)
    for i in range(num_qubits):
        state[i] = key[i]
    print(f"{name}'s Measured State:\t {np.array2string(state)}")

    return state

def extract_key(state1, state2, basis1, basis2, num_qubits, name1, name2):
    encryption_key1 = ''
    encryption_key2 = ''

    # we use two counters in order to verify the validity and similarity of the two keys
    acc_key = 0
    acc_qbit = 0
    perc = 0
    for i in range(num_qubits):
        if basis1[i] == basis2[i]: # when the bases are equal we extract the qubits to generate the key
            acc_key += 1
            encryption_key1 += str(state1[i])
            encryption_key2 += str(state2[i])
            if str(state1[i]) == str(state2[i]): # when the qubits are also equal the element is valid
                acc_qbit += 1 
                
    print('Correspondences between the bases: ',acc_key)
    print('Number of valid qbits for the encription key: ',acc_qbit)
    
    if acc_key == 0: # avoids "division by zero" types of errors 
        perc = 0
    else:
        perc = acc_qbit/acc_key # if there are more valid matches between the bases than between the qubits 
                                # themselves, our communication may have been intercepted; we need to define
                                # a safety threshold based on which we accept or discard the generated key
    print('Percentage of similarity between the keys: ', perc)
    
    if perc < 0.9:          # 90% safety threshold
        print("Key exchange has been tampered! Check for eavesdropper or try again")
        print(f"Alice's Key ({encryption_key1}) differs from Bob's Key ({encryption_key2})")
    else:
        print("Key exchange has been successful!")
        print(f"{name1}'s Key: {encryption_key1}")
        print(f"{name2}'s Key: {encryption_key2}")


In [75]:
# Main Code:

# Initializations:
num_qubits = 8  # message size for our communications
                # Note: 24 is the maximum number of qbits allowed before it is too complex for our machine to handle

# state: binary array denoting the state to be encoded; it will only be defined a
#        priori for alice, which is the sender, because the state is the logic string
#        which, after being encoded in alice's basis as qbits, will be used as message
# basis: binary array denoting the basis to be used for encoding, where 0 refers to
#        the Computational Basis and 1 to the Hadamard Basis

alice = QuantumCircuit(num_qubits) # starts the quantum circuit
alice_basis = np.random.randint(2, size=num_qubits)

# creates a binary array of size num_qubits filled with random elements 
alice_state = np.random.randint(2, size=num_qubits)

bob = QuantumCircuit(num_qubits)
bob_basis = np.random.randint(2, size=num_qubits)

eve = QuantumCircuit(num_qubits)
eve_basis = np.random.randint(2, size=num_qubits)

# we print the arrays we defined by converting them into strings first
print(f"Alice's State:\t {np.array2string(alice_state)}")
print(f"Alice's Bases:\t {np.array2string(alice_basis)}")
print(f"Bob's Bases:\t {np.array2string(bob_basis)}")

# Eavesdropper Flag:
E = 0   # IMPORTANT: through E we define whether an eavesdropper is present
        # on the public communication channel or not; we used this flag in order to be 
        # able to check the behavior in both cases (Eve is present or not present)

alice = encode(alice, alice_state, alice_basis) # we encode the Alice's logical state 
                                                # by converting it into qubits and initializing the quantum state of its corresponding 
                                                # quantum circuit to them; we follow Alice's basis for the conversion

if E == 0: # if Eve is not present

    # since we don't have two quantum computers and an optic fiber which would enable
    # us to send the qubits representing Alice's quantum state directly to Bob, we handle
    # the sending of the message by simply encoding Alice's quantum state onto Bob's 
    # quantum circuit (Bob simply recieves the message encoded in Alice's basis)

    bob = encode(bob,alice_state,alice_basis)
    bob = process(bob, bob_basis) # we then process the message recieved by decrypting it using bob's basis
    meas_state_b = measure_state(bob, num_qubits, 'Bob') # only at the end, we measure Bob's resulting 
    # quantum state in order to use it for computing the encryption key

else: # if Eve is present

    print(f"Eve's Bases:\t {np.array2string(eve_basis)}")

    # since Eve cannot clone nor measure Alice's message, and she doesn't know alice's basis, she 
    # can only try decrypting the message through a random basis of their own and then measure and 
    # send her own resulting quantum state to bob, encoded in her own basis, in order to pretend she 
    # didn't disrupt the comunication; in this way, though, her result will be probabilistic, and 
    # she must take a gamble in order not to be detected: only if her basis and Alice's basis coincide
    # she will surely be invisible and undetected

    eve = encode(eve, alice_state, alice_basis)
    eve = process(eve, eve_basis)
    meas_state_e = measure_state(eve, num_qubits, 'Eve')

    # we then carry out the same procedure we used for the Eve-less communication, but the message
    # is sent through the communication channel from Eve to Bob instead (and encoded in eve's basis)

    bob = encode(bob, meas_state_e, eve_basis)
    bob = process(bob, bob_basis)
    meas_state_b = measure_state(bob, num_qubits, 'Bob')

# finally, we assume Alice and Bob's respective bases are shared through the private communication 
# channel, and through them each entity extracts the cryptographic key from their own state 
extract_key(alice_state, meas_state_b, alice_basis, bob_basis, num_qubits, 'Alice', 'Bob')


Alice's State:	 [1 1 1 0 0 0 1 1]
Alice's Bases:	 [0 0 0 1 0 1 1 1]
Bob's Bases:	 [1 1 0 1 1 1 1 1]
Eve's Bases:	 [0 1 1 1 0 1 1 0]
Eve's Measured State:	 [1 0 1 0 0 0 1 0]
Bob's Measured State:	 [1 0 1 0 0 0 1 0]
Correspondences between the bases:  5
Number of valid qbits for the encription key:  4
Percentage of similarity between the keys:  0.8
Key exchange has been tampered! Check for eavesdropper or try again
Alice's Key (10011) differs from Bob's Key (10010)
